# Evaluation de modèle avec Fiftyone

## Importations et Initialisation

### Librairies

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
from utils import init_notebook, predictions
%aimport datasets, utils.predictions
from utils.predictions import * 

import fiftyone as fo
from transformers import DetrForObjectDetection,DetrImageProcessor
import sahi
from sahi import AutoDetectionModel
import torch
import pandas as pd
import sys
import os

from pathlib import Path

HOME = Path(os.getcwd()).parents[2]
HOME

print("torch available :",torch.cuda.is_available())
torch.cuda.empty_cache()

### Datasets

#### Roboflow

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="VsgwOHYfQtqRLRCUpPv7")
# project = rf.workspace("suttyworkspace").project("cassini_800")
# version = project.version(1)
# dataset = version.download("coco")

#### Fiftyone dataset

In [ ]:
dataset = fo.Dataset.from_dir(
    data_path=HOME/"resources/datasets/coco_datasets/cocass_auvergne_v2/images",  # path to the dataset
    labels_path=HOME/"resources/datasets/coco_datasets/cocass_auvergne_v2/detailed.json",   # path to the labels/annotations file
    dataset_type=fo.types.COCODetectionDataset, # the type of dataset to import
)
dataset

# Les annotations sont nommées en tant que détections, nous les renommons
#pour éviter les confusions
dataset.rename_sample_field("detections", "annotations")

## Inférence sur les données pour l'évaluation
Il y a plusieurs manières d'inférer sur les données. Soit en utilisant la méthode SAHI, ce qui est le plus adapté lorsqu'on prend en entrée une feuille entière.<br>
Pour des crops de cartes (nous prenons pour le moment 1200x1200 px) il n'est probablement pas nécessaire de faire des slices des images.

### Instanciation du modèle de détection

In [ ]:
# settings
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIDENCE_TRESHOLD = 0.5
IOU_TRESHOLD = 0.7

model_path=HOME/"resources/models/detr/detr_coco7_2800"

CHECKPOINT = 'facebook/detr-resnet-152'

image_processor = DetrImageProcessor.from_pretrained(CHECKPOINT)
model=DetrForObjectDetection.from_pretrained(model_path)    # load the model from the checkpoint as a DEtrForObjectDetection model pretrained by the weights of the model_path
#model = Detr(model)
model.to(DEVICE)

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type="huggingface",
    #config_path="../../resources/models/detr/detr_coco7_2800/config.json",
    model=model,
    processor=image_processor,
    confidence_threshold=0.7,
    device="cuda:0", # or 'cuda:0'
)
type(detection_model)

In [ ]:
for sample in dataset.iter_samples(progress=True, autosave=True):
    fo_predict_simple(detection_model,sample, 
        label_field="predictions", 
        overlap_height_ratio=0.6, overlap_width_ratio=0.6)

In [ ]:
session = fo.launch_app(dataset, auto=False)
session.open_tab()